# Assignment 2: Large Language Models for Text Classification

### CS 410/510 Large Language Models Fall 2024

#### Greg Witt

## Load Dataset

**Install Dependencies**

In [126]:
!pip install datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
85077.27s - pydevd: Sending message related to process being replaced timed-out after 5 seconds



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [19]:
from datasets import load_dataset
import random

# load the training set of tweets
ds_train = load_dataset("cardiffnlp/tweet_sentiment_multilingual", "english", split="train")
# the dataset labels {0 : negative, 1: neutral, 2: positive }
print(f"""tweet_sentiment_multilingual training dataset: 
    -----------------------------------------
        {ds_train} 
    """)
random_tweet_index = random.randint(0,1839)
print(f"""
    Random Tweet:
    {ds_train['text'][random_tweet_index]}

    Label: 
    {ds_train['label'][random_tweet_index] }
""")


tweet_sentiment_multilingual training dataset: 
    -----------------------------------------
        Dataset({
    features: ['text', 'label'],
    num_rows: 1839
}) 
    

    Random Tweet:
    Good to see @user new book available in hard copy.  Amazon Prime bringing it tomorrow. 

    Label: 
    2



In [20]:

# load the training set of tweets
ds_validation = load_dataset("cardiffnlp/tweet_sentiment_multilingual", "english", split="validation")
# the dataset labels {0 : negative, 1: neutral, 2: positive }
print(f"""tweet_sentiment_multilingual validation set: 
    -----------------------------------------
        {ds_validation}
    """)
random_tweet_index = random.randint(0,324)
print(f"""
    Random Tweet:
    {ds_validation['text'][random_tweet_index]}

    Label: 
    {ds_validation['label'][random_tweet_index] }
""")


tweet_sentiment_multilingual validation set: 
    -----------------------------------------
        Dataset({
    features: ['text', 'label'],
    num_rows: 324
})
    

    Random Tweet:
    Sorry for the lack of Minecraft my computer is broke so I can't play it on PC. May try to record Xbox 360 Minecraft till I get a new one. 

    Label: 
    1



## Load the Models

**Load Dependencies**

In [129]:

!pip install transformers torch

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
85086.12s - pydevd: Sending message related to process being replaced timed-out after 5 seconds



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


#### [Lama 3.2 1B](https://huggingface.co/meta-llama/Llama-3.2-1B) 



In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

llama_3_2_1B = "meta-llama/Llama-3.2-1B"

llama_3_2_1B_tokenizer = AutoTokenizer.from_pretrained(llama_3_2_1B)

llama_3_2_1B_model = AutoModelForCausalLM.from_pretrained(llama_3_2_1B)



In [102]:
from transformers import pipeline 

classifier = pipeline(model=llama_3_2_1B, task="text-classification")




Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [35]:
# tokenizer assigns the probability 
"Text: I hate it! \nSentiment (positive, negative, neutral):"

prompt_approach_two = f"""{tweet} Sentiment: """

In [36]:
# Text to classify
text = "Your text to classify here."

# Tokenize the text
inputs = tokenizer(text, return_tensors="pt")

# Get model predictions
outputs = model(**inputs)
logits = outputs.logits

# Determine the predicted class
predicted_class = torch.argmax(logits, dim=1).item()


NameError: name 'tokenizer' is not defined

In [ ]:
with torch.no_grad():
    outputs = model(**inputs, labels=inputs["input_ids"])
    log_probs = torch.nn.functional.log_softmax(outputs.logits, dim=-1)

In [20]:
Experiment #2 

1 prompt  2 positive , 2 negative, 2 neutral k = 2 {tweet} : Sentiment {?} =>

NameError: name 'Experiment' is not defined

In [23]:
# 3 
google_this if needed.
chain_of_thought = "think about this step by step this sound positive, negative, neurtal?\n"
f"{chain_of_thought} {tweet}"

# emotion based: pull on compassion strings.
"I need this for my job: classify as negative, positive or neutral"

SyntaxError: expected 'else' after 'if' expression (2972846960.py, line 2)

#### [Phi 3.5 Instruct]()

**Load The Model**

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# model name
phi_model = "microsoft/Phi-3.5-mini-instruct"

phi_3_5_tokenizer = AutoTokenizer.from_pretrained(phi_model)

phi_3_5_model = AutoModelForCausalLM.from_pretrained(phi_model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Experiments 1: Zero-Shot Inference 

### Create Prompt Iteration Function

In [4]:
prompt_template = "\"{}\" \n Sentiment (positive (2), negative (0), neutral (1)): "

### Llama 3.3 1B: Zero Shot Inference

31587
43324
60668


In [24]:
# iterate through the validation set
for tweet, label in zip(ds_validation['text'], ds_validation['label']):
    prompt_tweet = tweet
    gt_label = label
    # combine the prompt
    prompt = prompt_template.format(prompt_tweet)
    
    # generate the response
    prompt_ids = llama_3_2_1B_tokenizer.encode(prompt, return_tensors="pt")

    outputs = llama_3_2_1B_model.generate(
                        prompt_ids,
                        pad_token_id=llama_3_2_1B_tokenizer.eos_token_id,
                        max_new_tokens=1,
                    )
    
    generated_tokens = outputs[0]

    generated_response = llama_3_2_1B_tokenizer.decode(generated_tokens, skip_special_tokens=True)

    print(f"Generated Response: {generated_response}")
    print(f"Ground Truth: {gt_label}")

    # with torch.no_grad():
    #     outputs = llama_3_2_1B_model(**prompt_ids)
    #     logits = outputs.logits

    # predicted_class_id = torch.argmax(logits, dim=-1)

    # predicted_class_label = model.config.id2label[predicted_class_id.item()]


Generated Response: "@user @user I think after Charlie Hebdo the French did NOT react as the US did after 9/11. But they may do this time around. " 
 Sentiment (positive (2), negative (0), neutral (1)): 1
Ground Truth: 0
Generated Response: ""Interview with Devon Alexander \""""Speed Kills\"""" (VIDEO)  On Tuesday Oct 16th we had the privilege of catch up with... " 
 Sentiment (positive (2), negative (0), neutral (1)): 1
Ground Truth: 1
Generated Response: "Hold on... Sam Smith may do the theme to Spectre!? Dope!!!!!! #007 #SPECTRE #JamesBond " 
 Sentiment (positive (2), negative (0), neutral (1)): 2
Ground Truth: 2
Generated Response: "kingpin Saudi Arabia posted a record $98 billion budget deficit in 2015 due to the sharp fall in oil prices finance ministry said on Monday " 
 Sentiment (positive (2), negative (0), neutral (1)): 0
Ground Truth: 0
Generated Response: "Gonna watch Final Destination 5 tonight. I always leave the theater so afraid of everything. No huge escalators for sur

KeyboardInterrupt: 